In [16]:
import json
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold

# load json annotation = {dataset file 경로}
annotation='/opt/ml/dataset/train.json'
with open(annotation) as f: data = json.load(f)

var = [(ann['image_id'], ann['category_id']) for ann in data['annotations']]
X = np.ones((len(data['annotations']),1))
y = np.array([v[1] for v in var])
groups = np.array([v[0] for v in var])

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=411)

# x=data y=label
for train_idx, val_idx in cv.split(X, y, groups):
    print("TRAIN:", groups[train_idx])
    print(" ", y[train_idx])
    print(" TEST:", groups[val_idx])
    print(" ", y[val_idx])
    print(len(groups[train_idx]),len(groups[val_idx]))


TRAIN: [   0    1    1 ... 4881 4881 4881]
  [0 3 7 ... 7 1 7]
 TEST: [   6   13   13 ... 4882 4882 4882]
  [1 6 7 ... 0 1 1]
18633 4511
TRAIN: [   0    1    1 ... 4882 4882 4882]
  [0 3 7 ... 0 1 1]
 TEST: [   5    5    5 ... 4876 4876 4878]
  [7 0 0 ... 0 2 0]
18075 5069
TRAIN: [   0    3    3 ... 4882 4882 4882]
  [0 2 6 ... 0 1 1]
 TEST: [   1    1    1 ... 4877 4877 4880]
  [3 7 4 ... 7 7 0]
18631 4513
TRAIN: [   1    1    1 ... 4882 4882 4882]
  [3 7 4 ... 0 1 1]
 TEST: [   0    3    3 ... 4881 4881 4881]
  [0 2 6 ... 7 1 7]
18454 4690
TRAIN: [   0    1    1 ... 4882 4882 4882]
  [0 3 7 ... 0 1 1]
 TEST: [   4    4    4 ... 4868 4872 4872]
  [1 1 1 ... 2 4 6]
18783 4361


In [19]:
# check distribution
from collections import Counter
import pandas as pd

def get_distribution(y):
    y_distr = Counter(y)
    y_vals_sum = sum(y_distr.values())

    return [f'{y_distr[i]/y_vals_sum:.2%}' for i in range(np.max(y) +1)]

distrs = [get_distribution(y)]
index = ['training set']

for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X,y, groups)):
    train_y, val_y = y[train_idx], y[val_idx]
    train_gr, val_gr = groups[train_idx], groups[val_idx]

    assert len(set(train_gr) & set(val_gr)) == 0
    distrs.append(get_distribution(train_y))

    distrs.append(get_distribution(val_y))
    index.append(f'train - fold{fold_ind}')
    index.append(f'val - fold{fold_ind}')


categories = [d['name'] for d in data['categories']]
pd.DataFrame(distrs, index=index, columns = [categories[i] for i in range(np.max(y) + 1)])

,General trash,Paper,Paper pack,Metal,Glass,Plastic,Styrofoam,Plastic bag,Battery,Clothing
training set,17.14%,27.45%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
train - fold0,16.96%,27.45%,3.79%,4.13%,4.48%,12.61%,5.51%,22.28%,0.77%,2.02%
val - fold0,17.85%,27.42%,4.23%,3.70%,3.26%,13.15%,5.25%,22.77%,0.35%,2.02%
train - fold1,17.14%,27.24%,4.01%,3.98%,4.28%,12.77%,5.38%,22.32%,0.67%,2.20%
val - fold1,17.12%,28.17%,3.41%,4.26%,4.12%,12.51%,5.72%,22.57%,0.73%,1.38%
train - fold2,17.31%,27.39%,3.83%,4.08%,4.13%,12.80%,5.14%,22.68%,0.69%,1.94%
val - fold2,16.42%,27.68%,4.05%,3.88%,4.70%,12.36%,6.76%,21.12%,0.69%,2.35%
train - fold3,17.30%,27.47%,3.87%,4.06%,4.22%,12.63%,5.49%,22.39%,0.63%,1.95%
val - fold3,16.50%,27.36%,3.88%,3.99%,4.33%,13.07%,5.33%,22.30%,0.92%,2.32%
train - fold4,16.97%,27.67%,3.88%,3.97%,4.10%,12.77%,5.76%,22.20%,0.68%,2.00%


In [46]:
data_dir='/opt/ml/dataset/train.json'

with open(data_dir,'r') as f:
    train_json=json.loads(f.read())

images=train_json['images']
categories=train_json['categories']
annotations=train_json['annotations']

In [30]:
train_json["licenses"]

[{'id': 0,
  'name': 'CC BY 4.0',
  'url': 'https://creativecommons.org/licenses/by/4.0/deed.ast'}]

In [47]:
from collections import defaultdict
i=0
for train_idx, val_idx in cv.split(X, y, groups):
    '''
    print("TRAIN:", groups[train_idx])
    print(" ", y[train_idx])
    print(" TEST:", groups[val_idx])
    print(" ", y[val_idx])
    print(len(groups[train_idx]),len(groups[val_idx]))
    '''

    train_json = defaultdict(list)
    val_json = defaultdict(list)

    for image in images:
        if image['id'] in groups[train_idx]:
            train_json['images'].append(image)
        if image['id'] in groups[val_idx]:
            val_json['images'].append(image)


    for annotation in annotations:
        if annotation['image_id'] in groups[train_idx]:
            train_json['annotations'].append(annotation)
        if annotation['image_id'] in groups[val_idx]:
            val_json['annotations'].append(annotation)

    train_json['categories']=categories
    val_json['categories']=categories

    with open(f'stratify_dataset/train_fold_{i}.json','w') as f:
        json.dump(train_json,f,ensure_ascii=False,indent=4)
    with open(f'stratify_dataset/val_fold_{i}.json','w') as f:
        json.dump(val_json,f,ensure_ascii=False,indent=4)
    print(len(train_json['images']),len(val_json['images']))
    i+=1


    
    
    

3914 969
3906 977
3901 982
3902 981
3909 974
